<a href="https://colab.research.google.com/github/yukinaga/ai_stock_prediction/blob/main/section_3/03_svm_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIによる株価予測（SVM・分類版）（データの取得・学習・推論）
分類（上がるか下がるかを当てる）のタスクを適用します。  
正解ラベル: 変動率がプラスなら 1 (Up)、マイナスなら 0 (Down)。  

SVMの実装には、PyTorchではなく scikit-learn を使用します。

In [ ]:
# 必要なライブラリのインストールとインポート
# !pip install -q yfinance

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# --- 設定 ---
STOCK_CODE = '7203.T'  # 予測対象（例: トヨタ自動車）
START_DATE = '2020-01-01'
END_DATE = '2024-12-31'
WINDOW_SIZE = 15
TEST_RATIO = 0.2

# 1. データの取得と前処理
print("データを取得中...")
df = yf.download(STOCK_CODE, start=START_DATE, end=END_DATE, progress=False)

# 変動率（リターン）を計算
# 予測のために、元の変動率の値も保持しておきます
raw_returns = df['Close'].pct_change().dropna().values

# --- 分類タスク用のラベル作成 ---
# 変動率が 0より大きければ 1 (Up)、それ以外は 0 (Down)
labels = np.where(raw_returns > 0, 1, 0)

# 正規化（特徴量のみ）
scaler = StandardScaler()
# SVM用にデータを整形 (サンプル数, 1)
data_normalized = scaler.fit_transform(raw_returns.reshape(-1, 1))

# データセットの作成関数
def create_dataset(dataset, labels, raw_returns, window_size):
    X, y, y_returns = [], [], []
    for i in range(len(dataset) - window_size):
        # 特徴量: 過去30日間の変動率
        feature = dataset[i:i + window_size]
        # ターゲット: 翌日の「クラス」（0 or 1）
        target = labels[i + window_size]
        # シミュレーション用: 翌日の「実際の変動率」
        actual_ret = raw_returns[i + window_size]

        X.append(feature)
        y.append(target)
        y_returns.append(actual_ret)

    return np.array(X), np.array(y), np.array(y_returns)

X, y, y_returns = create_dataset(data_normalized, labels, raw_returns, WINDOW_SIZE)

# SVM（scikit-learn）は (サンプル数, 特徴量数) の2次元入力を期待するため、
# (サンプル数, 30, 1) -> (サンプル数, 30) に変形します
X_flatten = X.reshape(X.shape[0], -1)

# 学習用とテスト用に分割
split_index = int(len(X) * (1 - TEST_RATIO))
X_train, y_train = X_flatten[:split_index], y[:split_index]
X_test, y_test = X_flatten[split_index:], y[split_index:]

# シミュレーション用に実際の変動率も分割
_, y_returns_test = y_returns[:split_index], y_returns[split_index:]

# 2. SVMモデルの定義と学習
# RBFカーネル（非線形分離）を使用
print("SVMモデルを学習中...")
clf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
clf.fit(X_train, y_train)

print("学習完了。")

# 予測結果の評価 その1（正解率と混同行列）
分類タスクのため、評価指標は「RMSE（誤差）」ではなく「Accuracy（正解率）」と「混同行列（当たった数・外れた数の内訳）」になります。

In [ ]:
# 推論の実行
predictions = clf.predict(X_test)

# 1. 正解率 (Accuracy)
acc = accuracy_score(y_test, predictions)
print(f"テストデータの正解率 (Accuracy): {acc * 100:.2f}%")

# 2. 混同行列 (Confusion Matrix) の表示
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Down', 'Up'])

# プロット
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.title('Confusion Matrix (SVM Prediction)')
plt.show()

print("\n--- 解説 ---")
print("左上(True Negative): 下がると予測して、実際に下がった数")
print("右下(True Positive): 上がると予測して、実際に上がった数")
print("正解率が50%に近い場合、ランダム（コイントス）と変わらない性能であることを意味します。")

# 予測結果の評価 その2（投資シミュレーション）
分類モデルの場合、「予測が 1 (Up) なら買い、0 (Down) なら休む」という戦略になります。

In [ ]:
# 投資シミュレーション
initial_capital = 100000.0

# SVM戦略
svm_capital = [initial_capital]
current_svm_capital = initial_capital

# ガチホ戦略
hold_capital = [initial_capital]
current_hold_capital = initial_capital

# テスト期間の日数分ループ
for i in range(len(y_test)):
    # 修正ポイント: 明示的に float (数値) に変換して、配列になるのを防ぐ
    actual_return = float(y_returns_test[i])

    # モデルの予測（0:Down, 1:Up）
    prediction = predictions[i]

    # --- SVM戦略 ---
    # 「上がる(1)」と予測した時だけポジションを持つ
    if prediction == 1:
        current_svm_capital = current_svm_capital * (1 + actual_return)
    else:
        # 「下がる(0)」と予測したら現金保有（変動なし）
        pass

    svm_capital.append(current_svm_capital)

    # --- ガチホ戦略 ---
    current_hold_capital = current_hold_capital * (1 + actual_return)
    hold_capital.append(current_hold_capital)

# 結果の可視化
plt.figure(figsize=(14, 6))
plt.plot(svm_capital, label='AI Strategy (SVM Classification)', color='orange')
plt.plot(hold_capital, label='Buy & Hold Strategy', color='blue', linestyle='--')

plt.title('Investment Simulation: SVM Classification vs Buy & Hold')
plt.xlabel('Days')
plt.ylabel('Portfolio Value')
plt.legend()
plt.grid(True)
plt.show()

print(f"最終資産 (Buy & Hold): {hold_capital[-1]:.2f}")
print(f"最終資産 (AI Strategy - SVM): {svm_capital[-1]:.2f}")

if svm_capital[-1] > hold_capital[-1]:
    print("結果: SVMモデルが市場平均を上回りました。")
else:
    print("結果: SVMモデルは市場平均に及びませんでした。")